In [38]:
import numpy as np

In [39]:
def g(p, beta):
    """
    Implements the Potts model tree recursion.
    
    Inputs: 
    p: a (d x q) matrix, giving the marginal probabilities for each of the children. 
    d here is the number of children and q the number of colors.
    
    beta: the temperature.
    
    
    Output: a row vector in R^q, giving the marginal probabilities of the parent.
    """
    numerator = np.prod(1 - (1-beta) * p, axis=0)
    denominator = np.sum(numerator)
    return numerator / denominator

def J(p, beta, Psi):
    """
    Calculates the Jacobian of the tree recursion.
    
    Inputs:
    p: a (d x q) matrix, giving the marginal probabilities for each of the children.
    d here is the number of children and q the number of colors.

    beta: the temperature.
    
    Psi: (a python function) the derivative of the potential function. It's important 
    that this function is implemented in such a way that in can be applied elementwise
    to numpy arrays.
    
    Output: a (d x q x q) 3-tensor J, that represents the Jacobian of the tree recursion. 
    For each i \in [d], J_i is the Jacobian with respect to just the i'th child's marginal
    probabilities.
    """
    term1 = (1-beta) * ( np.outer(g(p, beta), g(p, beta)) - np.diag(g(p, beta)) )
    
    def to_apply(p_i):
        return np.diag(Psi(g(p,beta))) @ term1 @ np.diag(1/(1-(1-beta)*p_i)) @ np.diag(1/Psi(p_i))
    
    return np.apply_along_axis(to_apply, 1, p)

In [40]:
def Jacobian_test():
    """Numerical small example, with Psi = 1."""
    def Psi(x):
        """Written like this to compatibilize with numpy arrays."""
        return 0 * x + 1
    p = np.array([[1,0], [1,0]])
    betas = np.linspace(0.1, 2, 5)
    for b in betas:
        Jac = J(p, b, Psi)
        for i in range(len(Jac)):
            if not np.allclose(Jac[0], Jac[i]):
                print("Children jacobians not equal with equal marginals:")
                print(Jac[0])
                print(Jac[i])
                return False
        true_val = np.array([
            [-(1-b)*b/(1+b**2) + (1-b)*b**2*b/(1+b**2)**2, (1-b)*b**2 / (1+b**2)**2],
            [(1-b)*b / (1+b**2)**2, -(1-b)/(1+b**2) + (1-b)/(1+ b**2)**2]
        ])
        if not np.allclose(Jac[0], true_val):
            print("Jacobians J_i give the wrong value.")
            print("Obtained:")
            print(Jac[0])
            print("Truth:")
            print(true_val)
            return False
    return True

if not Jacobian_test():
    print("Test 1 failed.")
else:
    print("Test passed.")

Test passed.
